In [1]:
import os

from utils import read_metadata_without_fields, read_metadata_with_fields,  request
from os import getcwd
import pandas as pd
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

In [2]:
start_dir = getcwd()
model_sharing = read_metadata_without_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_metadata-sharing.tsv")
mol_sample = read_metadata_without_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_molecular_metadata-sample.tsv")
merged = model_sharing.merge(mol_sample, on="model_id", how="left")
merged = merged[merged.fillna(0)['sample_origin'] != 0]
#merged = merged[merged['sample_origin'] != "xenograft"]
merged = merged[["model_id", "sample_id", "sample_origin", "database_url", 'raw_data_url', "passage", 'host_strain_name', 
                 'host_strain_nomenclature', 'engrafted_tumor_collection_site', 'platform_id']]
merged = merged.drop_duplicates(['model_id', 'sample_id']).reset_index(drop=True)

In [ ]:
def parse_html(response):
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    data_rows = []
    for row in table.find_all('tr')[2:]:  # Skip the first two rows (headers and separator)
        data = [td.text.strip() for td in row.find_all('td')]
        if len(data)>0 and data[0].__contains__("General Model Information"):
            data_rows.append(data)
    m = re.search("MSI.*HLA.*\n.*\n.*", re.sub("\n+", "\n", data_rows[0][0]))
    if m:
        out = m.group()
        out = out.replace('MSI Status', 'MSI Status: ')
        out = out.replace("HLA Profile", "\nHLA Profile: ")
    else:
        out = ''
    if out.__contains__('MSI Status'):
        msi = out.split("\n")[0].split(" ")[2]
    else:
        msi = ''
    if out.__contains__("HLA Profile"):
        out = re.sub("\(s\)\n.*\n", "", out)
        out = re.sub("./.", "", out)
        hla = out.split('\n')[1].split(" ")[2]
    else:
        hla = ''
    return [msi, hla]

ms_sheet = read_metadata_with_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_molecular_metadata-sample.tsv")
immune_sheet = pd.DataFrame(columns=['Field', 'sample_id', 'marker_type', 'marker_name', 'marker_value', 'essential_or_additional_details', 'platform_id'])
types = ["MMR", "TMB","Mutations per mb", "Ploidy"]
pid = {"MMR": 'immune_mmr', "TMB": 'immune_tmb',"Mutations per mb": 'immune_mpm', "Ploidy": 'immune_ploidy'}
for i in tqdm(range(merged.shape[0]), "Fetch from PDMR"):
    mid = merged.iloc[i, 0]
    sid = merged.iloc[i, 1]
    so = merged.iloc[i, 2]
    link = merged.iloc[i, 3]
    rdu = merged.iloc[i, 4]
    response = request(link, False, 'get')
    dt = set()
    if response.text is not None:
        msi, hla = parse_html(response)
        if msi.__contains__("MSI"):
            msi = msi.replace("MSI-", "")
            ms_sheet = pd.concat([ms_sheet, pd.DataFrame([['', mid, sid, so, merged.iloc[i, 6], merged.iloc[i, 7], merged.iloc[i, 8], merged.iloc[i, 9], rdu, 'immune_MSI']],  columns=ms_sheet.columns)])
            row = [['', sid, 'Model Genomics', 'MSI', msi, '', 'immune_MSI']]
            immune_sheet = pd.concat([immune_sheet, pd.DataFrame(row,  columns=immune_sheet.columns)])
            dt.add('MSI')
        if hla != '':
            pattern = re.compile(r'([A-Z]\*\d{2}:\d{2})')
            hla_types = pattern.findall(hla)
            hla_a_types = ['HLA-' + hla_type for hla_type in hla_types if hla_type.startswith('A')]
            hla_b_types = ['HLA-' + hla_type for hla_type in hla_types if hla_type.startswith('B')]
            hla_c_types = ['HLA-' + hla_type for hla_type in hla_types if hla_type.startswith('C')]
            hla_a_string = ', '.join(hla_a_types)
            hla_b_string = ', '.join(hla_b_types)
            hla_c_string = ', '.join(hla_c_types)
            row = [['', sid, 'HLA type', 'HLA-A', hla_a_string, '', 'immune_HLA_type'], ['', sid, 'HLA type','HLA-B', hla_b_string, '', 'immune_HLA_type'], ['', sid, 'HLA type','HLA-C', hla_c_string, '', 'immune_HLA_type']]
            immune_sheet = pd.concat([immune_sheet, pd.DataFrame(row,  columns=immune_sheet.columns)])
            ms_sheet = pd.concat([ms_sheet, pd.DataFrame([['', mid, sid, so, merged.iloc[i, 6], merged.iloc[i, 7], merged.iloc[i, 8], merged.iloc[i, 9], rdu, 'immune_HLA_type']],  columns=ms_sheet.columns)])
            dt.add('HLA')
    for t in types:
        if t not in dt:
            dt.add(t)
            ms_sheet = pd.concat([ms_sheet, pd.DataFrame([['', mid, sid, so, merged.iloc[i, 6], merged.iloc[i, 7], merged.iloc[i, 8], merged.iloc[i, 9], rdu, pid[t]]],  columns=ms_sheet.columns)])
            row = [['', sid, 'Model Genomics', t, 'Not provided', '', pid[t]]]
            immune_sheet = pd.concat([immune_sheet, pd.DataFrame(row,  columns=immune_sheet.columns)])
            
immune_sheet.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_immunemarker-Sheet1_new.tsv", sep='\t', index=False)
ms_sheet.drop_duplicates()
ms_sheet.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_molecular_metadata-sample_2.tsv", sep='\t', index=False)

Fetch from PDMR:   0%|          | 4/4270 [00:03<1:04:24,  1.10it/s]

In [13]:
sample_ids = immune_sheet.sample_id.unique()
for s in sample_ids:
     types = ["MMR", "TMB","Mutations per mb", "Ploidy"]
     pid = {"MMR": 'immune_mmr', "TMB": 'immune_tmb',"Mutations per mb": 'immune_mpm', "Ploidy": 'immune_ploidy'}
     for t in types:
         row = ['', s, 'Model Genomics', t, 'Not provided', '', pid[t]]
         immune_sheet = pd.concat([immune_sheet, pd.DataFrame([row], columns=immune_sheet.columns)])
immune_sheet

,Field,sample_id,marker_type,marker_name,marker_value,essential_or_additional_details,platform_id
0,,764851-200-R,Model Genomics,MSI,Stable,,immune_MSI
0,,764851-200-R,HLA type,HLA-A,"HLA-A*01:01, HLA-A*23:01",,immune_HLA_type
1,,764851-200-R,HLA type,HLA-B,"HLA-B*08:01, HLA-B*35:08",,immune_HLA_type
2,,764851-200-R,HLA type,HLA-C,"HLA-C*04:01, HLA-C*07:01",,immune_HLA_type
0,,764851-200-R-W9NP10,Model Genomics,MSI,Stable,,immune_MSI
...,...,...,...,...,...,...,...
0,,476673-061-R-V1-organoid,Model Genomics,Ploidy,Not provided,,immune_ploidy
0,,487391-300-R-V3-organoid,Model Genomics,MMR,Not provided,,immune_mmr
0,,487391-300-R-V3-organoid,Model Genomics,TMB,Not provided,,immune_tmb
0,,487391-300-R-V3-organoid,Model Genomics,Mutations per mb,Not provided,,immune_mpm


In [14]:
immune_sheet.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/PDMR/PDMR_immunemarker-Sheet1_new.tsv", sep='\t', index=False)

In [15]:
model_sharing = read_metadata_without_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_metadata-sharing.tsv")
mol_sample = read_metadata_without_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_molecular_metadata-sample.tsv")

merged = model_sharing.merge(mol_sample, on="model_id", how="left")
merged = merged[merged.fillna(0)['sample_origin'] != 0]
merged = merged[["model_id", "sample_id", "sample_origin", "database_url", 'raw_data_url', "passage", 'host_strain_name', 
                 'host_strain_nomenclature', 'engrafted_tumor_collection_site', 'platform_id']]
merged = merged.drop_duplicates(['model_id', 'sample_id']).reset_index(drop=True)
# Assuming df is your DataFrame with 'sample_id' and 'passage_number' columns
merged['passage_number'] = pd.to_numeric(merged['passage'], errors='coerce')  # Convert passage_number to numeric
merged = merged.sort_values(by=['model_id', 'passage_number'])
merged = merged.drop_duplicates(subset='model_id', keep='last').reset_index(drop=True)

In [22]:
merged

,model_id,sample_id,sample_origin,database_url,raw_data_url,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,platform_id,passage_number
0,CRL-1003,RXF_1003-2005,xenograft,https://compendium.criver.com/compendium2/mode...,RXF_1003-2005.cel,10,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,10.0
1,CRL-1012,LXFA_1012-1139,xenograft,https://compendium.criver.com/compendium2/mode...,LXFA_1012-1139.cel,7,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,7.0
2,CRL-1022,LXFE_1022-1122,xenograft,https://compendium.criver.com/compendium2/mode...,LXFE_1022-1122.cel,7,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,7.0
3,CRL-1023,OVXF_1023-5712,xenograft,https://compendium.criver.com/compendium2/mode...,OVXF_1023-5712.cel,20,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,20.0
4,CRL-1034,CXF_1034-4446,xenograft,https://compendium.criver.com/compendium2/mode...,CXF_1034-4446.cel,7,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
468,CRL-SMTCA88,Oncotest-R0057,xenograft,https://compendium.criver.com/compendium2/mode...,NG-9398_LYXFDLBC_SMTCA88_R_1735_lib121944_4532...,5,NOG/NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,NaN,mutation_RNA_sequencing,5.0
469,CRL-SMTCA96,A2874-08,xenograft,https://compendium.criver.com/compendium2/mode...,A2874-08.CEL,6,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,copy_number_alteration_Affymetrix_SNP6.0,6.0
470,CRL-TH-001,A3559_51,xenograft,https://compendium.criver.com/compendium2/mode...,NaN,4,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,expression_Affymetrix_HGU133plus2,4.0
471,CRL-TH-005,CRL-R0215,xenograft,https://compendium.criver.com/compendium2/mode...,R_2188_TH_005_1.fq.gz|R_2188_TH_005_2.fq.gz,6,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,mutation_RNA_sequencing,6.0


In [29]:
def parse_html(response):
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')
    table = [t for t in table if t.text.__contains__('HLA')]
    hla = ''
    if len(table) > 0:
        data_rows = []
        for row in table[0].find_all('tr'):  # Skip the first two rows (headers and separator)
            data = [re.sub("\(.*", "", td.text.strip()) for td in row.find_all('td')]
            data_rows.append(data)
        if len(data_rows) > 0:
            data_rows = pd.DataFrame(data_rows).drop_duplicates()
            data_rows['HLA'] = data_rows[0] + ', ' + data_rows[1]
            hla = list(data_rows['HLA'])  
        else:
            hla = ''
    return hla

ms_sheet = read_metadata_with_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_molecular_metadata-sample.tsv")
immune_sheet = pd.DataFrame(columns=['Field', 'sample_id', 'marker_type', 'marker_name', 'marker_value', 'essential_or_additional_details', 'platform_id'])
for i in tqdm(range(merged.shape[0]), "Fetch from CRL"):
    mid = merged.iloc[i, 0]
    sid = merged.iloc[i, 1]
    so = merged.iloc[i, 2]
    link = merged.iloc[i, 3]
    rdu = merged.iloc[i, 4]
    passage = merged.iloc[i]['passage']
    host_strain_name = merged.iloc[i]['host_strain_name']
    host_strain_nomenclature = merged.iloc[i]['host_strain_nomenclature']
    engrafted_tumor_collection_site = merged.iloc[i]['engrafted_tumor_collection_site']
    
    response = request(link, False, 'get')
    if response.text is not None:
        hla_types = parse_html(response)
        if hla_types != '':
            row = []
            for hla in hla_types:
                ht = re.search("\, HLA-.*\*", hla).group()[2:-1]
                row.append(['', sid, 'HLA type', ht, hla, '', 'immune_HLA_type'])
            immune_sheet = pd.concat([immune_sheet, pd.DataFrame(row,  columns=immune_sheet.columns)]).reset_index(drop=True)
            ms_sheet = pd.concat([ms_sheet, pd.DataFrame([['', mid, sid, so, passage, host_strain_name, host_strain_nomenclature, engrafted_tumor_collection_site, '', 'immune_HLA_type']],  columns=ms_sheet.columns)]).reset_index(drop=True)

Fetch from CRL: 100%|██████████| 473/473 [04:10<00:00,  1.89it/s]


In [31]:
immune_sheet.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_immunemarker-Sheet1.tsv", sep='\t', index=False)
ms_sheet.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_molecular_metadata-sample.tsv", sep='\t', index=False)

In [30]:
ms_sheet

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,CRL-984,Oncotest-001,xenograft,9,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,Oncotest-001.pe1.fq.gz|Oncotest-001.pe2.fq.gz,mutation_Illumina_WES
...,...,...,...,...,...,...,...,...,...,...
2574,,CRL-SMTCA88,Oncotest-R0057,xenograft,5,NOG/NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,NaN,,immune_HLA_type
2575,,CRL-SMTCA96,A2874-08,xenograft,6,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,,immune_HLA_type
2576,,CRL-TH-001,A3559_51,xenograft,4,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,,immune_HLA_type
2577,,CRL-TH-005,CRL-R0215,xenograft,6,NMRI,Crl:NMRI-Foxn1<sup>nu</sup>,NaN,,immune_HLA_type


PDX_HLA_A_ALLELES
PDX_HLA_B_ALLELES
PDX_HLA_C_ALLELES


In [34]:
model_sharing = read_metadata_without_fields("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_metadata-sharing.tsv")

In [33]:
model_sharing

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,recommended,desirable,essential
4,,CBAB-10855-V2,CBAB-10855,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Intravenous,,immune_HLA_type
...,...,...,...,...,...,...,...,...,...,...
70,,DFBL-69487-V1,DFBL-69487,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,Intravenous,,immune_HLA_type
71,,DFBL-72516-R2,DFBL-72516,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,"Renal Capsule, Intravenous",,immune_HLA_type
72,,DFBL-86381-R1,DFBL-86381,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,"Renal Capsule, Intravenous",,immune_HLA_type
73,,DFTL-85005-V4,DFTL-85005,xenograft,0,NSG,NOD.Cg-Prkdc<sup>scid</sup> Il2rg<sup>tm1Wjl</...,"Renal Capsule, Intravenous",,immune_HLA_type


In [2]:
def parse_html(response, mid):
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the div containing histology images
    histo_images_div = soup.find('div', class_='row histo_images')
    if histo_images_div:
        # Find all image links within the div
        image_links = histo_images_div.find_all('a', class_='histo_image_for_zoom')
        headers = histo_images_div.find_all('h4')
        # Extract and print the href attribute of each link
        images = []
        for l in range(len(image_links)):
            image_href = image_links[l].get('href')
            header = headers[l].get_text()
            m = re.search("Passage: .*,", header)
            if m:
                passage = m.group()[9:-1]
                passage = re.sub('N\d*', '', passage)
            else:
                passage = ''
            m = re.search("Magnification: .*", header)
            if m:
                mag = m.group()[15:]            
            else:
                mag = ''
            images.append(['', mid, 'https://compendium.criver.com'+image_href, header, 'pdx', passage, mag, ''])
            #f"Image description: {header} \nHistology Image Link: https://compendium.criver.com{image_href}"
        return images
    else:
        return ''    


model_images = [
["#Description", "Unique identifier for all the PDXs derived from the same tissue sample","Link to the model image Description of the image on what was being imaged", "Type of sample being imaged Passage number imaging was performed.", "Passage 0 correspond to first engraftment (if this is not the case please define how passages are numbered).", "Magnification of the mode image Staining used for imaging the sample"],
["#Example", " J000077451", "https://www.ebi.ac.uk/biostudies/files/S-BIAD850/submission/J000077451/J000077451180207133606.tiff", "P2 tumor ID018 for PDX model J000077451 stained for H&E.", "pdx", "0", "20x","H&E"],
["#Format Requirements", "free alphanumerical", "URL", "free text", "pdx, pateint, organoid, cell line", "numeric", "free text", "free text"],
["#Essential?", "desirable", "desirable", "desirable", "desirable", "desirable", "desirable", "desirable"]]

model_images = pd.DataFrame(model_images, columns=["Field", "model_id", "URL", "description","sample_type","passage","magnification", "staining"])

for i in tqdm(range(model_sharing.shape[0]), "Fetch from CRL"):
    mid = model_sharing.iloc[i, 0]
    link = model_sharing.iloc[i, 6]
    response = request(link, False, 'get')
    row = parse_html(response, mid)
    if row != '':
        row = pd.DataFrame(row, columns=model_images.columns)
        model_images = pd.concat([model_images, row]).reset_index(drop=True)
model_images.to_csv("/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/CRL/CRL_metadata-model_image.tsv", sep='\t', index=False)

In [71]:
table = soup.find_all('table')
soup

,model_id,sample_id,id_x,attributes.mutations_per_mb,attributes.ploidy,attributes.msi_status,_merge
0,SIDM00001,SIDS00001,SIDM00001,35.5,,Stable,both
1,SIDM00827,SIDS00002,SIDM00827,,,,both
2,SIDM00002,SIDS00781,SIDM00002,34.97,3.184795,Stable,both
3,SIDM00003,SIDS00780,SIDM00003,138.0,2.888203,Stable,both
4,SIDM00004,SIDS00779,SIDM00004,,,,both
...,...,...,...,...,...,...,...
1545,SIDM01910,SIDS01725,SIDM01910,,,,both
1546,SIDM01970,SIDS01866,SIDM01970,,,,both
1547,SIDM01272,SIDS01192,SIDM01272,8.04,,Stable,both
1548,SIDM01959,SIDS01814,SIDM01959,3.48,,Stable,both


(3008, 10)


/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_62769/650031041.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immune_data['Field'] = ''
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_62769/650031041.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  immune_data['marker_value'] = immune_data[it]
/var/folders/_2/g5d3zf4s41g0tvlqkc3_8wjr0000gp/T/ipykernel_62769/650031041.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,Field,model_id,sample_id,sample_origin,passage,host_strain_name,host_strain_nomenclature,engrafted_tumor_collection_site,raw_data_url,platform_id
0,#Description,Unique identifier for all the PDXs derived fro...,Identifier of the sample from any patient tiss...,The orgin of the sample - xenograft (extracted...,Indicate the passage number of the sample wher...,"Host mouse strain name (e.g. NOD-SCID, NSG, et...",The full nomenclature form of the host mouse s...,The anatomical site from which the xenograft s...,Unique identify to platform metadata between s...,Unique identify to platform metadata between s...
1,#Example,CRC0228PR,CRC0228PRH0000000000D01000,xenograft,0,NOD SCID,NOD.Cg-Prkdcscid Il2rgtm1Wjl/SzJ,mammary glad,mutation1,mutation1
2,#Format Requirements,free alphanumerical,free alphanumerical,"xenograft, patient","numerical, or blank",follow strain name or Not Specified (if unknown),https://www.jax.org/jax-mice-and-services/cust...,free alphanumerical,NaN,NaN
3,#Essential?,essential,essential,essential,essential,essential,essential,desirable,desirable,essential
4,NaN,SIDM01291,SIDS01224,cell,NaN,NaN,NaN,NaN,EGAS00001002222,mutation_WGS_Illumina_NGS
...,...,...,...,...,...,...,...,...,...,...
3003,NaN,SIDM00151,SIDS00825,cell,NaN,NaN,NaN,NaN,EGAS00001000978,cna_WES_Illumina_NGS
3004,NaN,SIDM00282,SIDS00657,cell,NaN,NaN,NaN,NaN,EGAS00001000978,cna_WES_Illumina_NGS
3005,NaN,SIDM00168,SIDS00617,cell,NaN,NaN,NaN,NaN,NaN,cna_WES_Illumina_NGS
3006,NaN,SIDM01690,SIDS01630,cell,NaN,NaN,NaN,NaN,NaN,cna_WES_Illumina_NGS


(3015, 7)

In [80]:
import requests

# Define the target URL
url = 

# Create a session
session = requests.Session()

# Perform a request to get the initial session ID
response = session.get(url)

# Extract and print the initial session ID
initial_session_id = re.search('<script type="application/shiny-singletons">.*<' , response.text)
if initial_session_id:
    print(f"Initial Session ID: {initial_session_id.group()}")

,Field,sample_id,marker_type,marker_name,marker_value,essential_or_additional_details,platform_id
0,,SIDS00001,Model Genomics,Mutations per mb,35.5,,immune_mpm
1,,SIDS00781,Model Genomics,Mutations per mb,34.97,,immune_mpm
2,,SIDS00780,Model Genomics,Mutations per mb,138.0,,immune_mpm
3,,SIDS01009,Model Genomics,Mutations per mb,27.97,,immune_mpm
4,,SIDS00777,Model Genomics,Mutations per mb,145.29,,immune_mpm
...,...,...,...,...,...,...,...
3010,,SIDS00384,Model Genomics,MSI,Stable,Samples with an MSI score >=7 were marked as H...,immune_MSI
3011,,SIDS00380,Model Genomics,MSI,Stable,Samples with an MSI score >=7 were marked as H...,immune_MSI
3012,,SIDS01192,Model Genomics,MSI,Stable,Samples with an MSI score >=7 were marked as H...,immune_MSI
3013,,SIDS01814,Model Genomics,MSI,Stable,Samples with an MSI score >=7 were marked as H...,immune_MSI


In [8]:
session = requests.Session()

# Perform a request to get the initial session ID
response = session.get(url)

# Extract and print the initial session ID
initial_session_id = re.search('<script type="application/shiny-singletons">.*<' , response.text)
if initial_session_id:
    print(f"Initial Session ID: {initial_session_id.group()}")

In [8]:
response.cookies.keys()

In [8]:
response.content

In [8]:
resp = request(url, False, 'get')

In [8]:
resp.text

In [8]:
url2 = 'https://portal.pdxnetwork.org/session/add739c82ab207ed2c80be4b7e4b181525eb7a75/dataobj/dt_model_submission?w=&nonce=9dbc654ffde57b04'

In [8]:
request(url2, False, 'get')

In [8]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


download_dir = getcwd()
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--headless")  # Run Chrome in headless mode if desired
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# Navigate to the URL
driver.get(url)
try:
    # Specify the element you want to wait for (replace with the appropriate selector)
    element = WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.ID, "btn_about")))
    # Click the element
    element.click()
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "DataTables_Table_0_length"))
    )
    element.click()
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//option[@value='-1']"))
    )
    element.click()
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Download')]"))
    )
    element.click()
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'CSV')]"))
    )
    element.click()
    print("File downloaded successfully!")
except Exception as e:
    print("Error waiting for page to load:", e)    
sleep(15)
print("Page title:", driver.title)


In [8]:
driver.quit()

In [8]:
# Close the browser
download_dir
# https://portal.pdxnetwork.org/session/1e4830102cd9c61148244965750ffeee/dataobj/dt_model_submission?w=&nonce=2d3cc473dd9dc174

In [8]:
t = 'https://portal.pdxnetwork.org/session/177c56c551e2efcbd189ff17d30594d2/dataobj/dt_model_submission?w=&nonce=9f024178893b04fa'
request(t, False, 'post').text

In [8]:
from requests_html import HTMLSession

s = HTMLSession()
response = s.get(url)
response.html.render()

print(response)